In [3]:
# Install medcat
! pip install medcat==1.2.7

     |████████████████████████████████| 141 kB 5.4 MB/s 
  Using cached transformers-4.11.3-py3-none-any.whl (2.9 MB)
     |████████████████████████████████| 280 kB 48.4 MB/s 
     |████████████████████████████████| 831.4 MB 6.4 kB/s 
  Using cached pandas-1.3.4-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.3 MB)
  Using cached datasets-1.14.0-py3-none-any.whl (290 kB)
  Using cached spacy-3.1.3-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.9 MB)
     |████████████████████████████████| 24.1 MB 1.4 MB/s 
  Using cached jsonpickle-2.0.0-py2.py3-none-any.whl (37 kB)
     |████████████████████████████████| 177 kB 55.0 MB/s 
     |████████████████████████████████| 243 kB 49.6 MB/s 
  Using cached scipy-1.7.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (28.5 MB)
  Using cached elasticsearch-8.0.0-py3-none-any.whl (369 kB)
     |████████████████████████████████| 15.7 MB 40.4 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |██████████

In [4]:
from tokenizers import ByteLevelBPETokenizer
import gensim
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

In [5]:
DATA_DIR = "./data/"

In [6]:
!mkdir ./data
!mkdir ./models
!wget https://raw.githubusercontent.com/CogStack/MedCATtutorials/main/notebooks/introductory/data/noteevents.csv -P ./data/

--2022-02-15 14:57:32--  https://raw.githubusercontent.com/CogStack/MedCATtutorials/main/notebooks/introductory/data/noteevents.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7171226 (6.8M) [text/plain]
Saving to: ‘./data/noteevents.csv’

noteevents.csv      100%[===================>]   6.84M  --.-KB/s    in 0.08s   

2022-02-15 14:57:32 (81.0 MB/s) - ‘./data/noteevents.csv’ saved [7171226/7171226]



### Meta Annotations with MedCAT

To train meta-annotations (e.g. Experiencer, Negation...) we need two additional models:
- Tokenizer: to tokenize the text
- Embeddings: Word2Vec or any other type of embeddings that will be used for meta annotations. 

For meta-annotations we will use a custom BiLSTM model with simulated attention that works very well with sub-word tokenizers and embeddings creating using Word2Vec or BERT (for simplicity we will use w2v here). All of this is also available for download (check next tutorial) and we only need to rebuild the tokenizer/embeddings if our use-case is from a very specific domain. 

In [7]:
# To train the tokenizer we will use all the data we have from our dummy dataset.
df = pd.read_csv(DATA_DIR + "noteevents.csv")
df.head()

,Unnamed: 0,subject_id,chartdate,category,text
0,0,0,01/01/2086,Urology,"CHIEF COMPLAINT: , Blood in urine.,HISTORY OF ..."
1,1,0,01/01/2086,Emergency Room Reports,"CHIEF COMPLAINT: , Blood in urine.,HISTORY OF ..."
2,2,0,01/01/2086,General Medicine,"CHIEF COMPLAINT: , Blood in urine.,HISTORY OF ..."
3,3,0,01/01/2086,General Medicine,"CHIEF COMPLAINT:, Followup on hypertension an..."
4,4,0,01/01/2086,Consult - History and Phy.,"CHIEF COMPLAINT: , Blood in urine.,HISTORY OF ..."


In [8]:
# The tokenizers from huggingface require us to save all the text used for 
#training into one/multiple text files.
f = open(DATA_DIR + "tok_data.txt", 'w')
for text in df['text'].values:
  #We'll remove new lines, so that we have one document in one line
  text = text.strip().replace("\n", ' ')
  f.write(text.lower()) # Lowercase text to remove noise
  f.write("\n")
f.close()

In [9]:
# Create, train and save the tokenizer
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(DATA_DIR + "tok_data.txt")
tokenizer.save("./models/bbpe")

In [10]:
# Now we tokenize all the text we have and train word2vec
f = open(DATA_DIR + "tok_data.txt", 'r')
# Note that if you have a very large dataset, use iterators that
#read the text line by line from the file, do not load the whole file
#into memory.
data = []
for line in f:
  data.append(tokenizer.encode(line).tokens)
w2v = Word2Vec(data, vector_size=300, min_count=1)

In [11]:
# Check is word2vec trained, Ġ - for this tokenizer denotes start of word (a space)
w2v.wv.most_similar('Ġcancer')

[('Ġcolon', 0.7445815205574036),
 ('Ġmetastatic', 0.7424247860908508),
 ('Ġmesothelioma', 0.7303459644317627),
 ('Ġcarcinoma', 0.6841891407966614),
 ('Ġfather', 0.6742925643920898),
 ('Ġaugmentation', 0.6699813604354858),
 ('Ġbreast', 0.6673422455787659),
 ('Ġdied', 0.6376239657402039),
 ('Ġmother', 0.6104230284690857),
 ('Ġfamily', 0.6059643626213074)]

In [12]:
# Now we just have to create the embeddings matrix
embeddings = []
for i in range(tokenizer.get_vocab_size()):
  word = tokenizer.id_to_token(i)
  if word in w2v.wv:
    embeddings.append(w2v.wv[word])
  else:
    # Assign a random vector if the word was not frequent enough to receive
    #an embedding
    embeddings.append(np.random.rand(300))

In [13]:
# Save the embeddings
np.save(open("./models/embeddings.npy", 'wb'), np.array(embeddings))